# APSG tutorial - tensors in structural geology
[Ondrej Lexa](https://petrol.natur.cuni.cz/~ondro/)  
Institute of Petrology and Structural Geology  
Faculty of Science, Charles University  
[lexa.ondrej@gmail.com](mailto:lexa.ondrej@gmail.com)

In [ ]:
%pylab inline

In [ ]:
from apsg import *
plt.rcParams['figure.figsize'] = (9, 6)

This package provides three classes to manipulate stress and strain tensors.
 * `DefGrad` fo deformation gradient tensor
 * `VelGrad` for velocity gradient tensor
 * `Stress` for stress tensor

## Deformation gradient tensor

The homogeneous deformation (rotation and strain) without translation could be described as:

$$ \begin{bmatrix}x \\ y \\ z\end{bmatrix}=\begin{bmatrix} a & b & c \\ d & e & f \\ g & h & i \end{bmatrix}\begin{bmatrix}X \\ Y \\ Z\end{bmatrix}$$

$$\vec{x}=\mathbf{F}\vec{X}$$
where $ \mathbf{F} $ is so called **deformation gradient**.

You can create instance of `DefGrad` by class method `from_comp`:
*Note the setup of coordinate system in APSG**

In [ ]:
s = StereoNet(grid=True)
s.vector(Vec3([1,0,0]), label='X')
s.vector(Vec3([0,1,0]), label='Y')
s.vector(Vec3([0,0,1]), label='Z')
s.show()

In [ ]:
F = DefGrad.from_comp(xy=1) # simple shear in xy plane in x-direction
F

In [ ]:
u = Lin(90, 0)
v = u.transform(F)
v

Note the change of length of deformed vector

In [ ]:
abs(u), abs(v)

We can apply deformation gradient on `Group` as well. So let's create set of linear features and transform them by deformation gradient $F$.

In [ ]:
# randn_lin method create pseudonormaly distributed lines around mean direction
g = Group.randn_lin(mean=Lin(90, 0), N=100, sig=40, name='Original')
gd = g.transform(F).uv  # Normalize vectors after deformation
gd.name =  'Deformed'

In [ ]:
StereoNet(g, gd);

We can check the change in intensity of preffered orientation using e.g. strength of orientation tensor

In [ ]:
g.ortensor.strength, gd.ortensor.strength

Or visuzlize on Frabric plot

In [ ]:
FabricPlot(g, gd);

## Velocity gradient tensor
Velocity gradients are absolutely essential to analyses involving path dependent materials and are useful to better understanding deformations.

The **velocity gradient** is to velocities what the deformation gradient is to displacements.The velocity gradient is used as a measure of the rate at which a material is deforming.

The relative velocity between the points is

$$d\mathbf{v} = {\partial{\mathbf{v}} \over \partial{x}}dx = \mathbf{L}dx$$

If the **velocity gradient** is constant, the above equation can be solved exactly using *matrix exponential*:

$$\mathbf{F}=\mathrm{e}^{\mathbf{L}t}$$

We can use these equation to calculate deformation gradient for any intermediate deformation. When deformation in time $t=1$ is defined by $\mathbf{F}$, than for any time $t=[0,1]$:
$$\mathbf{F}(t)=\mathrm{e}^{ln(\mathbf{F})t}$$

You can create instance of `VelGrad` by class method `from_comp`:
*Note the setup of coordinate system in APSG**

In [ ]:
L = VelGrad.from_comp(yy=-1, xx=0.8, zz=0.2)
L

To calculate deformation gradient at any given time, we can use `defgrad` method of `VelGrad`:

In [ ]:
L.defgrad() # Default value for time is 1

In [ ]:
L.defgrad(time=0.5)

We can create simple code to demonstrate fabric evolution during deformation. We will start with uniformly distributed lines and we will progressively apply deformation.

In [ ]:
g = Group.uniform_lin()    # Uniformly distributed linear features
ots = []
f = FabricPlot()
for t in linspace(0, 3, 20):
    F = L.defgrad(time=t)
    gd = g.transform(F).uv
    f.plot(gd.ortensor, 'ko', label=None)
f.show()

## Example case - transpression
Your task is to explore fabric evolution during transpression for different angles of convergence ($\alpha$).

Transpression and transtension were terms introduces by Harland (1971) to define deformation that involves both transcurrent (strike-slip) movement along a zone and compression or extension across it. Sanderson & Marchini (1984) produced a strain model for transpression, and the concept has subsequently been applied in a variety of tectonic settings over a wide range of scales. Transpression is modelled by the simultaneous application of a transcurrent shear and horizontal shortening orthogonal to a block, with no lateral stretch.

<img src="images/trasnpression.png" width="40%">